In [1]:
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import gc
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import tensorflow as tf
# import tensorflow.compat.v1 as tf
from tensorflow.keras.layers import BatchNormalization, Dense, MaxPool2D, Conv2D, Dropout, GlobalAveragePooling2D, Flatten, Input, Concatenate, GlobalMaxPool2D, Lambda
from tensorflow.keras import Sequential, Model
from tensorflow.keras.applications import ResNet101
from itertools import combinations
import keras
import tensorflow.keras.backend as K
from scipy import spatial
import os
from PIL import Image
import h5py
import math
import numpy as np
import gc
import torch
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
for dirname, _, filenames in os.walk('/kaggle/input/inputdata/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
f = open("../input/caltech-birds-2011-dataset/CUB_200_2011/classes.txt")
class_label_dict = {}
for line in f:
    items = line.split()
    class_label_dict[items[1]] = int(items[0])-1


In [3]:
global_train_labels = []
train_filenames = []
f = open("/kaggle/input/classeslist/trainvalclasses.txt")
for line in f:
    global_train_labels.append(class_label_dict[line.strip()])
    train_filenames.append(line.strip())
    
global_test_labels = []
test_filenames = []
f = open("/kaggle/input/classeslist/testclasses.txt")
for line in f:
    global_test_labels.append(class_label_dict[line.strip()])
    test_filenames.append(line.strip())

In [4]:
# loop on labels -> feature -> array of weights. Create text features array

    
f = open("/kaggle/input/class-attributes/class_attribute_labels_continuous.txt")
class_features_list = []
for line in f:
    values = [float(numeric_string) for numeric_string in line.split()]
    class_features_list.append(values)
    
class_features_list = np.array(class_features_list)
# len(labels)

In [5]:
def load_data(directory, folder_names):
    
    data =[]
    filename_list = []
    
    for folder_name in folder_names:
        for dirname, _, filenames in os.walk(os.path.join(directory, folder_name)):
            for filename in filenames:
                img = np.asarray(Image.open(os.path.join(dirname, filename)).convert('RGB').resize((256,256)))
                data.append(img)
                filename_list.append(os.path.join(dirname, filename))
    
    classes = [filename.split('/')[-2] for filename in filename_list]

    labels = [class_label_dict[cls] for cls in classes]
    
    return np.array(data), np.array(labels)


In [6]:
train_data, train_labels = load_data('/kaggle/input/caltech-birds-2011-dataset/CUB_200_2011/images', train_filenames)
test_data, test_labels = load_data('/kaggle/input/caltech-birds-2011-dataset/CUB_200_2011/images', test_filenames)

print("Train data shapes: ", train_data.shape, train_labels.shape)
print("Test data shapes: ", test_data.shape, test_labels.shape)

Train data shapes:  (8821, 256, 256, 3) (8821,)
Test data shapes:  (2967, 256, 256, 3) (2967,)


In [7]:
train_text_features = []
training_phase = np.ones(len(train_labels))
for label in train_labels:
    train_text_features.append(class_features_list[label])
    
train_text_features = np.array(train_text_features)

test_text_features = []
testing_phase = np.zeros(len(test_labels))
for label in test_labels:
    test_text_features.append(class_features_list[label])
    
test_text_features = np.array(test_text_features)

In [8]:
global_train_labels = np.array(global_train_labels)
train_labels_inds = np.arange(len(global_train_labels))

train_labels_inds_dict = dict(zip(global_train_labels, train_labels_inds))


global_test_labels = np.array(global_test_labels)
test_labels_inds = np.arange(len(global_test_labels))

test_labels_inds_dict = dict(zip(global_test_labels, test_labels_inds))

In [9]:
train_weights_matrix = []

for i in range(len(global_train_labels)):
    train_weights_matrix.append(class_features_list[global_train_labels[i]])
    
train_weights_matrix = np.array(train_weights_matrix)
    
test_weights_matrix = []

for i in range(len(global_test_labels)):
    test_weights_matrix.append(class_features_list[global_test_labels[i]])
    
test_weights_matrix = np.array(test_weights_matrix)

In [10]:
final_train_labels = []

for i in range(len(train_labels)):
    final_train_labels.append(train_labels_inds_dict[train_labels[i]])
    
final_train_labels = np.array(final_train_labels)
    
final_test_labels = []

for i in range(len(test_labels)):
    final_test_labels.append(test_labels_inds_dict[test_labels[i]])

final_test_labels = np.array(final_test_labels)

In [11]:
train_data_inds = np.random.permutation(len(train_data))

train_data_shuffled = train_data[train_data_inds]
final_train_labels_shuffled = final_train_labels[train_data_inds]
train_text_features_shuffled = train_text_features[train_data_inds]

In [12]:
A_sparse = sparse.csr_matrix(class_features_list)

similarities = cosine_similarity(A_sparse)

In [13]:
train_weights_matrix_numpy = torch.nn.functional.normalize(torch.tensor(train_weights_matrix)).numpy()
test_weights_matrix_numpy = torch.nn.functional.normalize(torch.tensor(test_weights_matrix)).numpy()

In [14]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0.0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class ClassAverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, n_labels):
        self.reset(n_labels)

    def reset(self,n_labels):
        self.n_labels = n_labels
        self.acc = np.zeros(n_labels)
        self.cnt = np.array([1e-8]*n_labels)
        self.pred_prob = []

    def update(self, val, cnt, pred_prob):
        self.acc += val
        self.cnt += cnt
        self.avg = 100*self.acc.dot(1.0/self.cnt).item()/self.n_labels
        self.pred_prob += pred_prob
        #print ('pred',len(self.pred_prob))

def accuracy(output_vec, target, n_labels):
    """Computes the precision@k for the specified values of k"""
    output = output_vec

    batch_size = target.shape[0]
    pred = output.argsort()[:,-1]
    class_accuracy = np.zeros(n_labels)
    class_cnt = np.zeros(n_labels)
    prec = 0.0
    pred_prob = []
    for i in range(target.shape[0]):
        t = target[i]
        pred_prob.append(output[i][t])

        if pred[i] == tf.cast(t, tf.int32):
            prec += 1
            class_accuracy[t] += 1
        class_cnt[t] += 1
    return prec*100.0/batch_size, class_accuracy, class_cnt, pred_prob

In [16]:
class CustomLayer(tf.keras.layers.Layer):

    def __init__(self, train_weights, test_weights):
        super(CustomLayer, self).__init__()
        self.train_weights = tf.transpose(train_weights)
        self.test_weights = tf.transpose(test_weights)
        
    def train_step(self, are_features, acse_features):
        are_output = tf.matmul(are_features, self.train_weights)
        acse_output = tf.matmul(acse_features, self.train_weights)
        are_output = tf.keras.activations.relu(are_output)
        acse_output = tf.keras.activations.relu(acse_output)
        return are_output, acse_output
    
    def test_step(self, are_features, acse_features):
        are_output = tf.matmul(are_features, self.test_weights)
        acse_output = tf.matmul(acse_features, self.test_weights)
        are_output = tf.keras.activations.relu(are_output)
        acse_output = tf.keras.activations.relu(acse_output)
        return are_output, acse_output


    def call(self, are_features, acse_features, is_training):
        return tf.cond(tf.equal(is_training, 1),
                       lambda: self.train_step(are_features, acse_features),
                       lambda: self.test_step(are_features, acse_features))
    
class CustomPreprocessing(tf.keras.layers.Layer):

    def __init__(self):
        super(CustomPreprocessing, self).__init__()
        self.random_crop = tf.keras.layers.experimental.preprocessing.RandomCrop(224, 224)
        self.random_flip = tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal')
        self.center_crop = tf.keras.layers.experimental.preprocessing.CenterCrop(224, 224)
        self.mean_tensor = tf.constant([0.406, 0.456, 0.485], name="mean_tensor")
        self.std_tensor = tf.constant([0.225, 0.224, 0.229], name="std_tensor")
        self.preproc_layer = Lambda(lambda x: (tf.cast(x, tf.float32) - self.mean_tensor) / self.std_tensor)
        
    def train_preprocess(self, image_features):
        image_features = self.random_crop(image_features)
        image_features = self.random_flip(image_features)
        image_features = self.preproc_layer(image_features)
        return image_features
    
    def test_preprocess(self, image_features):
        image_features = self.center_crop(image_features)
        image_features = self.preproc_layer(image_features)
        return image_features

    def call(self, image_features, is_training):
        return tf.cond(tf.equal(is_training, 1),
                       lambda: self.train_preprocess(image_features),
                       lambda: self.test_preprocess(image_features))

class MyModel(tf.keras.Model):
    def __init__(self, k, alpha, cmps):
        super(MyModel, self).__init__()
        self.resnet_base = ResNet101(include_top=False, input_shape=(224,224,3), weights="imagenet")
        self.resnet = Model(inputs=self.resnet_base.inputs, outputs=self.resnet_base.get_layer("conv5_block3_3_bn").output)
        self.flatten = Flatten()
        self.custom_preprocess = CustomPreprocessing()
        self.conv_channels = 2048
        self.threshold = alpha
        self.parts = k
        self.map_size = 7
        self.pool2d = MaxPool2D(pool_size=self.map_size, strides=self.map_size)
        self.conv2d = Conv2D(self.parts, 1, kernel_initializer=tf.keras.initializers.orthogonal)
        self.resnet_features = None
        self.resnet_feature_shapes = None
        self.conv_bilinear = Conv2D(cmps, 1, kernel_initializer=tf.keras.initializers.orthogonal)#, kernel_initializer=tf.keras.initializers.ConvolutionDeltaOrthogonal)
        self.coef = self.map_size * self.map_size
        self.p_linear = Dense(312, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal)
        self.b_linear = Dense(312, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal)
        self.attr_linear = Dense(312, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal)
        self.dropout_p = Dropout(0.4)
        self.dropout_b = Dropout(0.4)
        self.dropout_attr = Dropout(0.4)
        self.batchnorm = BatchNormalization()
        self.batchnorm_bilinear = BatchNormalization()
        self.shared_train_init = tf.constant_initializer(tf.keras.backend.l2_normalize(train_weights_matrix, axis=1).numpy())
        self.dense_se_1 = Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal)
        self.dense_se_2 = Dense(2048, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal)
        self.gap = GlobalAveragePooling2D()
        
        self.image_tower_are = Sequential(
            [
                Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal),
                Dropout(0.15),
                Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal),
                Dropout(0.15),
                Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal),
                Dropout(0.15),
                Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal),
                Dropout(0.15),
                Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal),
            ]
        )
        self.image_tower_acse = Sequential(
            [
                Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal),
                Dropout(0.15),
                Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal),
                Dropout(0.15),
                Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal),
                Dropout(0.15),
                Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal),
                Dropout(0.15),
                Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal),
            ]
        )
        self.text_tower = Sequential(
            [
                Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal),
                Dropout(0.15),
                Dense(512, use_bias=False, kernel_initializer=tf.keras.initializers.orthogonal),
            ]
        )

    def call(self, input1, input2, input3):
        input1 = self.custom_preprocess(input1, input3[0])

        feat = self.resnet(input1)
        feat = tf.keras.activations.relu(feat)
        
        # ==== Squeeze and Excitation Block ==== #
        # ==== Squeeze Step ==== #
        w = self.gap(feat)

        # ==== Excitation Step ==== #
        w = self.dense_se_1(w)
        w = tf.keras.activations.relu(w)
        w = self.dense_se_2(w)
        w = tf.keras.activations.sigmoid(w)
        w = tf.expand_dims(w, 1)
        w = tf.expand_dims(w, 1)
        w = tf.broadcast_to(w, [tf.shape(feat)[0], tf.shape(feat)[1], tf.shape(feat)[2], tf.shape(feat)[3]])
        feat = tf.math.multiply(feat, w)

        self.resnet_features = feat
        self.resnet_feature_shapes = tf.shape(feat)
        
        # ==== Self Attention ==== #
        att_weight = self.conv2d(feat)
        att_weight = tf.keras.activations.sigmoid(att_weight)
        
        batch = tf.shape(att_weight)[0]
        height = tf.convert_to_tensor(att_weight.shape[1])
        width = tf.convert_to_tensor(att_weight.shape[2])
        channels = tf.convert_to_tensor(att_weight.shape[3])
        
        global_max_value = tf.math.reduce_max(tf.reshape(att_weight, [batch, -1]), axis=1)
        local_max_value = tf.math.reduce_max(tf.reshape(att_weight, [batch, -1, self.parts]), axis=1)

        global_max_value = tf.broadcast_to(tf.expand_dims(global_max_value, axis=-1), shape=[batch, channels])
        threshold_value = self.threshold * global_max_value

        mask = tf.broadcast_to(tf.expand_dims(tf.expand_dims(tf.cast(tf.math.greater_equal(local_max_value, threshold_value), dtype=tf.float32), axis=1), axis=1),
                               shape=[batch, height, width, self.parts])

        attention_weights = tf.math.multiply(att_weight, mask)
        attention_weights = tf.transpose(attention_weights, perm=[3, 0, 1, 2])
        attention_weights = tf.expand_dims(attention_weights, axis=2)
        attention_weights_shape = tf.shape(attention_weights)
        attention_weights = tf.broadcast_to(attention_weights, shape=[attention_weights_shape[0], attention_weights_shape[1], self.conv_channels, attention_weights_shape[3], attention_weights_shape[4]])
        attention_weights = tf.transpose(attention_weights, perm=[0, 1, 3, 4, 2])
        features = tf.broadcast_to(tf.expand_dims(self.resnet_features, axis=0), [self.parts, self.resnet_feature_shapes[0], self.resnet_feature_shapes[1], self.resnet_feature_shapes[2], self.resnet_feature_shapes[3]])
        Y = tf.math.multiply(attention_weights, features)
        
        # ==== ARE Network ==== #
        are_output = tf.map_fn(lambda vec: self.pool2d(vec), Y)
        are_output = tf.squeeze(are_output, [2,3])
        are_output = tf.transpose(are_output, [1,0,2])
        weights_batch = tf.shape(are_output)[0]
        weights_channel = are_output.shape[1]
        weights_h = are_output.shape[2]
        are_output = tf.reshape(are_output, (weights_batch, weights_channel*weights_h))


        # ==== ACSE Network ==== #
        
        Y_shape = tf.shape(Y)
        Y_temp = tf.reshape(Y, shape=[Y.shape[0], Y_shape[1], Y.shape[2]*Y.shape[3], Y.shape[4]])
        self.bilinear = self.conv_bilinear(self.resnet_features)
        batch = tf.shape(self.bilinear)[0]
        h = self.bilinear.shape[1]
        w = self.bilinear.shape[2]
        channels = self.bilinear.shape[3]
        X = tf.reshape(self.bilinear, shape=[batch, h*w, channels])
        X = tf.transpose(X, [0,2,1])
        acse_output = tf.map_fn(lambda vec: (tf.einsum('ikj,ijl -> ikl', X, vec) / self.coef), Y_temp)
        acse_output_shape = tf.shape(acse_output)
        acse_output = tf.reshape(acse_output, shape=[acse_output.shape[0], acse_output_shape[1],
                                                     acse_output.shape[2]*acse_output.shape[3]])
        acse_output = tf.math.reduce_max(acse_output, axis=0)

        are_feat = self.dropout_p(self.p_linear(are_output))
        acse_feat = self.dropout_b(self.b_linear(acse_output))
        
        # ==== Image Tower ==== #
        are_feat_shared_space = self.image_tower_are(are_output)
        acse_feat_shared_space = self.image_tower_acse(acse_output)
        
        # ==== Text Tower ==== #
        attribute_train_shared_space = self.text_tower(train_weights_matrix_numpy)
        attribute_test_shared_space = self.text_tower(test_weights_matrix_numpy)
        input_text_features = self.text_tower(input2)
        
        are_feat, acse_feat = CustomLayer(attribute_train_shared_space, attribute_test_shared_space)(are_feat_shared_space, acse_feat_shared_space, input3[0])

        return are_feat, acse_feat

    def model(self):
        inp = Input(shape=(256,256,3))
        inp1 = Input(shape=(150))
        inp2 = Input(shape=(312))
        inp3 = Input(shape=())
        return Model(inputs=(inp, inp1, inp2, inp3), outputs=self.call(inp, inp2, inp3))

In [18]:
batch_size = 32

c_loss=tf.keras.losses.CategoricalCrossentropy()
def classification_loss(combined_output, labels):
    combined_output = tf.keras.activations.softmax(combined_output)
    return c_loss(labels, combined_output)

# ARE with SE Block and projections onto Shared Space

In [19]:
model_obj = MyModel(10, 1, 20)
model = model_obj.model()
learning_rate = 1e-3

cls_loss = classification_loss(model.outputs[0], model.inputs[1])

model.add_loss(cls_loss)

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, decay=0.0005)
model.compile(optimizer=optimizer, metrics=["accuracy"])

for layer in model.layers:
    if "custom_layer" in layer.name:
        layer.trainable = False
        print(layer.trainable)

171450368/171446536 [==============================] - 2s 0us/step


In [23]:
top1 = AverageMeter()
class_avg = ClassAverageMeter(50)
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        top1.reset()
        class_avg.reset(50)
        predictions = model.predict((test_data, final_test_labels_cat, test_text_features, testing_phase))
        print("Predictions Using Average result")
        avg_output = predictions[0]
        prec1,class_acc,class_cnt,prec_prob = accuracy(avg_output, final_test_labels, 50)

        top1.update(prec1, avg_output.shape[0])
        class_avg.update(class_acc,class_cnt,prec_prob)

        print("Epocs = ", i, " Top1 = ", top1.avg, " Class Avg = ", class_avg.avg)
        del predictions
        gc.collect()

drop = 20
def sgdr(epoch):
    lr = learning_rate*(0.5 ** (epoch // drop))
    return lr

reduce_lr = tf.keras.callbacks.LearningRateScheduler(sgdr, verbose=True)
on_epoch_callback = CustomCallback()

training_phase = np.ones(shape=len(train_data), dtype=np.int64)
final_train_labels_shuffled_cat = tf.keras.utils.to_categorical(final_train_labels_shuffled, num_classes=150)
final_test_labels_cat = tf.keras.utils.to_categorical(final_test_labels, num_classes=50)

model.fit((train_data_shuffled, final_train_labels_shuffled_cat, train_text_features_shuffled, training_phase),
            final_train_labels_shuffled_cat, epochs=100, batch_size=32,
            callbacks=[reduce_lr, on_epoch_callback])



gc.collect()


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0007930434120840454.
Epoch 1/100
276/276 [==============================] - ETA: 0s - loss: 4.0868 - sequential_3_accuracy: 0.0016 - sequential_4_accuracy: 0.0028 - sequential_5_accuracy: 0.0020 - custom_layer_1_accuracy: 0.0966 - custom_layer_1_1_accuracy: 0.0054Predictions Using Average result
Epocs =  2966  Top1 =  30.940343781597573  Class Avg =  30.915215562574357
276/276 [==============================] - 103s 373ms/step - loss: 4.0868 - sequential_3_accuracy: 0.0016 - sequential_4_accuracy: 0.0028 - sequential_5_accuracy: 0.0020 - custom_layer_1_accuracy: 0.0966 - custom_layer_1_1_accuracy: 0.0054

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0007965496096580481.
Epoch 2/100
276/276 [==============================] - ETA: 0s - loss: 2.1225 - sequential_3_accuracy: 0.0016 - sequential_4_accuracy: 0.0024 - sequential_5_accuracy: 0.0018 - custom_layer_1_accuracy: 0.3967 - custom_layer_1_1_accuracy: 0.

892

# ACSE with SE Block and projections onto Shared Space

In [ ]:
model_obj = MyModel(10, 1, 20)
model = model_obj.model()
learning_rate = 1e-3

cls_loss = classification_loss(model.outputs[1], model.inputs[1])

model.add_loss(cls_loss)

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, decay=0.0005)
model.compile(optimizer=optimizer, metrics=["accuracy"])

for layer in model.layers:
    if "custom_layer" in layer.name:
        layer.trainable = False
        print(layer.trainable)

In [20]:
top1 = AverageMeter()
class_avg = ClassAverageMeter(50)
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        top1.reset()
        class_avg.reset(50)
        predictions = model.predict((test_data, final_test_labels_cat, test_text_features, testing_phase))
        print("Predictions Using Average result")
        avg_output = predictions[1]
        prec1,class_acc,class_cnt,prec_prob = accuracy(avg_output, final_test_labels, 50)

        top1.update(prec1, avg_output.shape[0])
        class_avg.update(class_acc,class_cnt,prec_prob)

        print("Epocs = ", i, " Top1 = ", top1.avg, " Class Avg = ", class_avg.avg)
        del predictions
        gc.collect()

drop = 20
def sgdr(epoch):
    lr = learning_rate*(0.5 ** (epoch // drop))
    return lr

reduce_lr = tf.keras.callbacks.LearningRateScheduler(sgdr, verbose=True)
on_epoch_callback = CustomCallback()

training_phase = np.ones(shape=len(train_data), dtype=np.int64)
final_train_labels_shuffled_cat = tf.keras.utils.to_categorical(final_train_labels_shuffled, num_classes=150)
final_test_labels_cat = tf.keras.utils.to_categorical(final_test_labels, num_classes=50)

model.fit((train_data_shuffled, final_train_labels_shuffled_cat, train_text_features_shuffled, training_phase),
            final_train_labels_shuffled_cat, epochs=100, batch_size=32,
            callbacks=[reduce_lr, on_epoch_callback])



gc.collect()



Epoch 00001: LearningRateScheduler reducing learning rate to 0.0007996838804887793.
Epoch 1/100
276/276 [==============================] - ETA: 0s - loss: 4.7072 - sequential_accuracy: 0.0016 - sequential_1_accuracy: 0.0019 - sequential_2_accuracy: 1.1337e-04 - custom_layer_accuracy: 0.0054 - custom_layer_1_accuracy: 0.0441Predictions Using Average result
Epocs =  2966  Top1 =  25.14324233232221  Class Avg =  25.21478277383492
276/276 [==============================] - 104s 377ms/step - loss: 4.7072 - sequential_accuracy: 0.0016 - sequential_1_accuracy: 0.0019 - sequential_2_accuracy: 1.1337e-04 - custom_layer_accuracy: 0.0054 - custom_layer_1_accuracy: 0.0441

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0007826823021137196.
Epoch 2/100
276/276 [==============================] - ETA: 0s - loss: 2.4055 - sequential_accuracy: 0.0017 - sequential_1_accuracy: 0.0011 - sequential_2_accuracy: 1.1337e-04 - custom_layer_accuracy: 0.0078 - custom_layer_1_accuracy: 0.3684Pred

892